<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/135_%E9%9B%BB%E5%AD%90%E3%83%A1%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

電子メール
==========

電子メールの仕組み
------------------

### SMTP

SMTP（Simple Mail Transfer Protocol）は、IETF において標準化された電子メール送信・転送のためのプロトコルである。通常、TCP のポート番号 25 を使用する。

SMTP では、クライアントがサーバーに TCP 接続を確立するとただちにサーバー・クライアント間に SMTP セッションが確立され、その後、両者の間で対話型でコマンドやそれに対する応答やメールがやり取りされる。

| コマンド | 意味 | 説明 |
|:---|:---|:---|
| `HELO` | 挨拶 | クライアントがサーバーに自分のドメイン名を名乗る。古い仕様 |
| `EHLO` | 挨拶 | `HELO` の拡張版。サーバーは対応する拡張機能を一覧で返す |
| `MAIL FROM` | 送信者指定 | 送信メールの差出人のメールアドレスを指定 |
| `RCPT TO` | 宛先指定 | 最終的な宛先のメールアドレスを指定。複数回送信して複数宛先も指定可能 |
| `DATA` | メール本文開始 | メール本文入力を開始。本文終了時は `.` のみの行で終了 |
| `QUIT` | 接続終了 | セッションを終了する |

`MAIL FROM` コマンドの指定を**エンベロープ From**（Envelope From）といい、`RCPT TO` コマンドの指定を**エンベロープ To**（Envelope To）という。

サーバーからの応答コードは、3 桁の数字からなり、おおまかな分類が以下となる:

| 応答コード | 意味 |
|:--:|:---|
| 2xx | 成功 |
| 3xx | コマンドは受け入れられたが追加の情報を待っている |
| 4xx | 一時的エラー |
| 5xx | 永続的エラー |

代表的な応答コードは以下のとおり:

| 応答コード | 意味 | 説明 |
|:--:|:---|:---|
| **220** | サービス準備完了 | サーバーが接続を受け付けた |
| **221** | 接続終了 | QUIT 後などで接続を終了 |
| **250** | 成功 | コマンドが正常に受理された |
| **251** | ユーザー転送 | メールを他のアドレスに転送 |
| **354** | データ開始 | DATA コマンド後にメール本体を送信できる状態 |
| **421** | サービス利用不可 | サーバーが一時的に利用不可。接続を切断 |
| **450** | メールボックス使用不可 | 一時的な問題で宛先が利用できない |
| **451** | ローカルエラー | サーバーの一時的エラー |
| **452** | 容量不足 | サーバーのリソース不足で処理できない |
| **500** | 構文エラー | コマンド構文が不正 |
| **501** | パラメータエラー | コマンドの引数が不正 |
| **502** | コマンド未実装 | サーバがサポートしていないコマンド |
| **503** | コマンド順序エラー | コマンドの順番が不正 |
| **504** | パラメータ未実装 | サーバが指定されたパラメータをサポートしていない |
| **550** | 宛先拒否 | アカウントが存在しないか、拒否されている |
| **551** | ユーザー転送不可 | 宛先ユーザーが転送先に存在しない |
| **552** | ストレージ不足 | メールボックスがいっぱいなど |
| **553** | 宛先アドレス不正 | メールアドレスが不正 |
| **554** | トランザクション失敗  | メール送信拒否。迷惑メール対策などで返ることが多い |

TCP 接続確立後、SMTP はテキストベースで、次のようなやり取りをする：

``` text
    foo.com                                                 example.com
┏━━━━━━┓                                          ┏━━━━━━┓
┃            ┃               220 foo.com ESMTP Postfix  ┃            ┃
┃            ┃←――――――――――――――――――――┃            ┃
┃            ┃  HELO example.com                        ┃            ┃
┃            ┃――――――――――――――――――――→┃            ┃
┃            ┃          250 foo.com greets example.com  ┃            ┃
┃            ┃←――――――――――――――――――――┃            ┃
┃            ┃  MAIL FROM:<bob@example.com>             ┃            ┃
┃            ┃――――――――――――――――――――→┃            ┃
┃            ┃                                  250 Ok  ┃            ┃
┃            ┃←――――――――――――――――――――┃            ┃
┃            ┃  RCPT TO:<alice@foo.com>                 ┃            ┃
┃            ┃――――――――――――――――――――→┃            ┃
┃            ┃                                  250 Ok  ┃            ┃
┃            ┃←――――――――――――――――――――┃            ┃
┃クライアント┃  DATA                                    ┃  サーバー  ┃
┃            ┃――――――――――――――――――――→┃            ┃
┃            ┃     354 End data with <CR><LF>.<CR><LF>  ┃            ┃
┃            ┃←――――――――――――――――――――┃            ┃
┃            ┃  ...メール本体...                        ┃            ┃
┃            ┃――――――――――――――――――――→┃            ┃
┃            ┃  .                                       ┃            ┃
┃            ┃――――――――――――――――――――→┃            ┃
┃            ┃                                  250 Ok  ┃            ┃
┃            ┃←――――――――――――――――――――┃            ┃
┃            ┃  QUIT                                    ┃            ┃
┃            ┃――――――――――――――――――――→┃            ┃
┃            ┃                                 221 Bye  ┃            ┃
┃            ┃←――――――――――――――――――――┃            ┃
┗━━━━━━┛                                          ┗━━━━━━┛
```

SMTP で、接続したサーバーがメールの最終的な宛先ではなかった場合は、そのサーバーは宛先ドメインを調べ、自分の管理するドメインでなければ、適切な宛先サーバーに SMTP で転送する。宛先サーバーがメールを受信して、ユーザーごとに設定されたメールボックスに保管する。

### ESMTP・SMTPS

SMTP に追加された拡張は、**ESMTP**（SMTP Service Extensions）と総称される。

接続先のサーバーが ESMTP に対応しているかどうかは、`EHLO` コマンドを送ることによって確認することができる。

もともとの SMTP には認証機能も暗号化もなく、スパムメールの送信に悪用されたため、ほとんどのインターネットサービスプロバイダーはポート番号 25 をブロックする。このため、25 番ポートは主にサーバー間のメール転送に使用されている。

ユーザー認証の仕組みを導入する拡張は、**SMTP-AUTH**（SMTP Authentication）と呼ばれる。SMTP-AUTH では、送信前にユーザー名とパスワードなどで本人確認を行い、正規の利用者だけがメールを送信できるようにする。パスワードをそのまま送るのは危険なので、暗号化通信のための拡張コマンド **STARTTLS** も導入されている。これらの拡張機能のために、通常、TCP のポート番号 587 が使用される。

クライアントは、サーバーが利用可能な拡張を確認してから、拡張コマンド `STARTTLS` を送り、「これ以降は TLS で暗号化された通信に切り替える」ことをサーバーに伝える。通常は、TLS 暗号化通信に切り替えられた後に、ユーザー認証を行う。

ユーザー認証と TLS 暗号化に関係する拡張コマンド、応答コードは以下のとおり:

| ESMTP コマンド | 意味 | 説明 |
|:---|:---|:---|
| `AUTH` | ユーザー認証 | 認証メカニズムを指定する。`LOGIN` ならサーバーはユーザー名とパスワードを順に要求する。`PLAIN` なら一度にユーザー名・<br /><br />パスワードを送信する（サーバーの要求を待たない）。やり取りは互いに Base64 で送信する |
| `STARTTLS` | 暗号化 | これ以降は TLS で暗号化された通信に切り替える |

| 応答コード | 意味 | 説明 |
|:--:|:---|:---|
| **235** | 認証成功 | クライアントの資格情報が正しく、認証が成功した |
| **334** | 続きを送ってよい | サーバーがクライアントに「認証データを送信せよ」と促している状態 |
| **454** | 一時的に認証できない | サーバー側の制限による認証失敗 |
| **530** | 認証が必要 | まだAUTHしていない状態で送信を試みた |
| **535** | 認証失敗 | ユーザー名またはパスワードが無効 |

ESMTP では、次のようなやり取りをする：

``` text
    foo.com                                                           example.com
┏━━━━━━┓                                                    ┏━━━━━━┓
┃            ┃          220 mail.example.com ESMTP Service Ready  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃  EHLO client.example.org                           ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃     250-mail.example.com Hello client.example.org  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃                              250-AUTH LOGIN PLAIN  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃                                      250 STARTTLS  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃  STARTTLS                                          ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃                            220 Ready to start TLS  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃クライアント┃┌────────────────────────┐┃  サーバー  ┃
┃            ┃│TLS ハンドシェイク開始                          │┃            ┃
┃            ┃│以降のやり取りは実際には TLS で暗号化されている │┃            ┃
┃            ┃└────────────────────────┘┃            ┃
┃            ┃  AUTH LOGIN                                        ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃                                  334 VXNlcm5hbWU6  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃  Ym9iQGV4YW1wbGUuY29t                              ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃                                  334 UGFzc3dvcmQ6  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃  cGFzc3dvcmQxMjM=                                  ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃                     235 Authentication successful  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃  MAIL FROM:<bob@example.com>                       ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃┌────────────────────────┐┃            ┃
┃            ┃│以後省略                                        │┃            ┃
┗━━━━━━┛└────────────────────────┘┗━━━━━━┛
```

`EHLO` コマンドに対する応答コード `250` で利用可能な拡張が次々に通知される。`250-` のように応答コードがハイフン付きの場合、まだ続きの応答があることを示す。

認証のやり取りを Base64 デコードすると

  * `VXNlcm5hbWU6` → `Username:`
  * `Ym9iQGV4YW1wbGUuY29t` → `bob@example.com`
  * `UGFzc3dvcmQ6` → `Password:`
  * `cGFzc3dvcmQxMjM=` → `password123`

**SMTPS**（SMTP over TLS）は、通信開始時から TLS で暗号化される SMTP 派生版である。最初から暗号化するので、25 番ポートや 587 番ポートを使用できず（465 番ポートを使用する）、また、クライアントとサーバーの両方が対応していないと通信を開始できない。当然のことながら、SMTPS では `STARTTLS` コマンドは使わない。

### メール書式

電子メールの書式も、IETF により標準化されている。IETF の RFC 5322 は、電子メールの構造、ヘッダーの書式のほか、本文の表現方法（文字コードや行の折り返しなど）も定めている。

電子メールの構造は、次のようになる:

``` text
┏━━━━━━━━━━━━━━━━━━━┓
┃┌─────────────────┐┃
┃│                                  │┃
┃│        ヘッダーフィールド        │┃
┃│                                  │┃
┃└─────────────────┘┃
┃ （空行; CR+LF）                      ┃
┃┌─────────────────┐┃
┃│                                  │┃
┃│           ボディ（本文）         │┃
┃│                                  │┃
┃└─────────────────┘┃
┗━━━━━━━━━━━━━━━━━━━┛
```

ヘッダーフィールドの形式は、次のとおり。

``` text
<フィールド名>: <フィールド値>[; <パラメータ>=<パラメータ値>]
```

主なヘッダーフィールドは次のとおり（フィールド名の大文字と小文字は区別されない）:

| フィールド名 | 説明 |
|:---|:---|
| `From` | 差出人のメールアドレス |
| `To` | 宛先のメールアドレス |
| `Cc` | カーボンコピー（宛先以外にも参考として同じ内容のメールを送ること）の宛先 |
| `Bcc` | ブラインドカーボンコピー（カーボンコピーに似ているが宛先が画面上に表示されない）の宛先 |
| `Date` | メールの送信日時（RFC 5322 形式） 例: `Date: Sat, 06 Sep 2025 10:30:00 +0900` |
| `Subject` | 件名 |
| `Message-ID` | メールを識別するためのユニークな文字列（RFC 5322 形式） 例: `<03@123456.123456>` |
| `Reply-To` | 返信先アドレス |
| `MIME-Version` | MIME のバージョン（通常 `1.0`） |
| `Content-Type` | 本文の MIME タイプを指定する |
| `Content-Language` | 本文の言語を指定する（`ja-JP`, `en-US` など）。ほとんど使われていない |
| `Content-Transfer-Encoding` | 本文のエンコード方式（`7bit`, `base64` など） |
| `Received` | メールを受信したサーバーが付加する |

ヘッダーフィールドは、主にメールクライアントがメールの表示などに利用するもので、`From` ヘッダーと `To` ヘッダーは実際の配送元と配送先を表しているとは限らない。例えば、ブラインドカーボンコピーで送信する場合、宛先を `To` ヘッダーには設定しないことで、受信者に他の宛先を見せずに送信することができる。実際の配送元と配送先はエンベロープ From とエンベロープ To で知ることができる。

電子メールは、サーバーが 7bit の文字しか扱えない時代に発明されたことから、現在でも電子メールの文字コードを 7bit のエンコーディング（ASCII）とする慣習が残っている。

ヘッダーフィールドとメールの本文は、空行で区切られる。

メールの本文は、複数のパート（部分）に分割することができる。この形式を**マルチパート**（multipart）と呼ぶ。マルチパートでない形式は**シングルパート**（single part）と呼ぶ。

マルチパートはパートごとに個別のヘッダーフィールドを持ち、異なる種類のコンテンツ（テキスト、HTML、画像、添付ファイルなど）を組み合わせられる。本体の MIME タイプは次のようになる。

  * `Content-Type: multipart/alternative`  
→ 内容が同じプレーンテキスト版と HTML 版のように、形式は違っても内容は同じであることを示す
  * `Content-Type: multipart/mixed`  
→ 個々のパートはそれぞれ異なるものであることを示す（本文＋添付ファイルなど）
  * `Content-Type: multipart/related`  
→ 関連するものであることを示す（HTML本文＋埋め込み画像など）

マルチパートの場合、行頭から `--` で始まる特別な文字列だけを含む行で各パートが区切られる。MIME タイプの `boundary` パラメータで、 `--` の後ろに追加される文字列を指定する。この文字列は、メール本文中に含まれることはほとんどない、ランダムでユニークなものが使われる。

各パートのヘッダーフィールドには、コンテンツにより以下のものが含まれる。

| フィールド名 | 説明 |
|:---|:---|
| `MIME-Version` | MIME のバージョン（通常 `1.0`） |
| `Content-Type` | パートのコンテンツの MIME タイプを指定する |
| `Content-Language` | パートのコンテンツの本文の言語を指定する。ほとんど使われていない |
| `Content-Transfer-Encoding` | パートのコンテンツのエンコード方式（`7bit`, `base64` など） |
| `Content-Disposition` | `attachment` ならファイルとしてダウンロードされるべき添付ファイルであることを示し、`inline` ならメール本文中に画像が埋め<br /><br />込まれることを示す。`attachment` の場合、メールクライアントが表示するファイル名をパラメータとして `filename` で指定する |
| `Content-ID` | パートのコンテンツを識別するためのユニークな文字列（RFC 5322 形式） |

パートのヘッダーフィールドの `Content-Type` で MIME タイプを `multipart` とすることで、マルチパートをネストすることができる。

### IMAP

**IMAP**（Internet Message Access Protocol）は、メールサーバー上の電子メールをクライアントから操作するためのプロトコルであり、IETF において標準化されている。いくつかのバージョンや派生版が存在するが、現在広く使用されているのは、**IMAP4** と **IMAPS**（IMAP over SSL）である。

IMAP では、メールの状態を管理するために「フラグ」と呼ばれる属性が使われる。

| フラグ | 意味 | 説明
|:---|:---|:---|
| `\Seen` | 既読フラグ | メールが読まれたことを示す |
| `\Answered` | 返信済みフラグ | メールに返信したことを示す |
| `\Flagged` | 重要フラグ | 特別な注意を払うべきメールであることを示す |
| `\Deleted` | 削除フラグ | メールが削除予定であることを示す |
| `\Draft` | 下書きフラグ | メールが下書きであることを示す |
| `\Recent` | 新着フラグ | 現在のセッションで初めてこのメールボックスに到着したメールであることを示す |

IMAP4 は通常、TCP のポート番号 143 を使用する。

主な IMAP4 コマンドは以下の通り。

| コマンド | 意味 |
|:---|:---|
| `CAPABILITY` | サーバーがサポートしている機能を問い合わせる |
| `STARTTLS` | これ以降は TLS で暗号化された通信に切り替える |
| `LOGIN ユーザー名 パスワード` | 認証を行う。通常はプレーンテキスト認証（TLS 利用が前提） |
| `LOGOUT` | セッションを終了する |
| `LIST 参照 パターン"`	| メールボックスの一覧を取得する。`参照` はフォルダ名か空文字列 `""` を指定する。`パターン` は一覧を取得<br /><br />したいメールボックス名のパターン。ワイルドカードが `"*"` 使える |
| `SELECT メールボックス名` | 指定したメールボックスを選択し、アクセス対象とする |
| `EXAMINE メールボックス名` | `SELECT` と同様だが、読み取り専用で開く |
| `FETCH メッセージ番号 取得データ種類` | 指定したメッセージ番号のメールから、指定した種類のデータを取得する |
| `STORE メッセージ番号 フラグ操作 フラグ` | メールのフラグを設定する |
| `COPY メッセージ番号 宛先メールボックス` | メールを別のフォルダーにコピーする |
| `SEARCH 条件` | 条件に合うメールを検索する |
| `CREATE メールボックス名` | 新しいメールボックス（フォルダ）を作成する |
| `DELETE メールボックス名` | メールボックスを削除する |
| `SUBSCRIBE` / `UNSUBSCRIBE` | メールボックスの購読／購読解除を行う |
| `NOOP` | 何もしないが、サーバーの応答を確認したり、新着チェックを行うのに使う |
| `CHECK` | メールボックスの状態を更新（新着反映など） |
| `EXPUNGE` | 削除フラグが設定されたメールを完全に削除する |

`FETCH` コマンドによるメッセージ取得で使用可能な「取得データ種類」は以下の通り。

| 取得データ種類 | 意味 |
|:---|:---|
| `BODY[]` | メッセージの本文を、ヘッダーを除いて取得する |
| `BODY[HEADER]` | メッセージのヘッダー部分全体を取得する |
| `BODY[HEADER.FIELDS (フィールド名1 フィールド名2 ...)]` | 特定のヘッダーフィールドのみを取得する。例: `(BODY[HEADER.FIELDS (FROM SUBJECT DATE)])` |
| `BODY[TEXT]` | メッセージの本文全体（ヘッダーを除く）を取得する |
| `BODY.PEEK[... ]` | `BODY` オプションと同じであるが、メッセージの既読フラグを設定しない。例: `BODY.PEEK[HEADER]` |
| `RFC822` | メッセージ全体を、RFC 822 形式で取得する。`BODY[HEADER]` と `BODY[TEXT]` を同時に取得する<br /><br />のと同じ |
| `FLAGS` | メッセージに設定されているフラグを取得する |
| `INTERNALDATE` | IMAP サーバーがメッセージを受信した日時を取得する |
| `UID` | メッセージの一意な ID（UID）を取得する。UID はメッセージ番号と異なり不変 |

`SEARCH` コマンドによる検索で使用可能な「条件（キー）」は以下の通り。

基本的な検索条件:

| 条件 | 意味 |
|:---|:---|
| `ALL` | すべてのメッセージを検索する |
| `ANSWERED` | `\Answered` フラグが設定されているメッセージを検索する |
| `UNANSWERED` | `\Answered` フラグが設定されていないメッセージを検索する |
| `DELETED` | `\Deleted` フラグが設定されているメッセージを検索する |
| `UNDELETED` | `\Deleted` フラグが設定されていないメッセージを検索する |
| `DRAFT` | `\Draft` フラグが設定されているメッセージを検索する |
| `UNDRAFT` | `\Draft` フラグが設定されていないメッセージを検索する |
| `FLAGGED` | `\Flagged` フラグが設定されているメッセージを検索する |
| `UNFLAGGED` | `\Flagged` フラグが設定されていないメッセージを検索する |
| `SEEN` | `\Seen` フラグが設定されているメッセージを検索する |
| `UNSEEN` | `\Seen` フラグが設定されていないメッセージを検索する |
| `RECENT` | `\Recent` フラグが設定されているメッセージを検索する |
| `OLD` | `\Recent` フラグが設定されていないメッセージを検索する |
| `NEW` | `RECENT` かつ `UNSEEN` なメッセージを検索する |

ヘッダーや本文の検索:

| 条件 | 意味 |
|:---|:---|
| `BCC "string"` | `BCC` フィールドに `"string"` を含むメッセージを検索する |
| `CC "string"` | `CC` フィールドに `"string"` を含むメッセージを検索する |
| `FROM "string"` | `FROM` フィールドに `"string"` を含むメッセージを検索する |
| `TO "string"` | `TO` フィールドに `"string"` を含むメッセージを検索する |
| `SUBJECT "string"` | `SUBJECT` フィールドに `"string"` を含むメッセージを検索する |
| `BODY "string"` | 本文に `"string"` を含むメッセージを検索する |
| `TEXT "string"` | ヘッダーまたは本文に `"string"` を含むメッセージを検索する |
| `HEADER "field-name" "string"` | 特定のヘッダーフィールドに `"string"` を含むメッセージを検索する |

日付による検索:

| 条件 | 意味 |
|:---|:---|
| `BEFORE "date"` | 指定した日付より前にメールボックスに保存されたメッセージを検索する |
| `ON "date"` | 指定した日付にメールボックスに保存されたメッセージを検索する |
| `SINCE "date"` | 指定した日付以降にメールボックスに保存されたメッセージを検索する |
| `SENTBEFORE "date"` | 指定した日付より前に送信されたメッセージを検索する |
| `SENTON "date"` | 指定した日付に送信されたメッセージを検索する |
| `SENTSINCE "date"` | 指定した日付以降に送信されたメッセージを検索する |

サイズによる検索:

| 条件 | 意味 |
|:---|:---|
| `LARGER n` | `n` バイトより大きいメッセージを検索する |
| `SMALLER n` | `n` バイトより小さいメッセージを検索する |

論理演算子:

| 条件 | 意味 |
|:---|:---|
| `NOT "search-key"` | 指定した条件に一致しないメッセージを検索する |
| `OR "search-key1" "search-key2"` | 2 つの条件のうち、少なくともどちらか一方にマッチするメッセージを検索する |
| `"search-key1" "search-key2"` | 複数の検索条件をスペースで区切ると、自動的に論理積（AND）として扱われる |

検索条件は丸括弧で囲むことができる。

主な IMAP4 応答コードは以下の通り。

| 応答コード | 意味 |
|:---|:---|
| `OK` | 要求が正常に処理された |
| `NO` | コマンドが拒否された（例: 権限不足、無効な操作など） |
| `BAD` | コマンド構文が不正 |
| `PREAUTH` | 接続時にすでに認証済みであることを示す |
| `BYE` | サーバーがセッションを終了する |

IMAP4 の通信では、各コマンドの先頭に「タグ」を付けて送信する。サーバーの応答もこのタグに対応して返されるため、どのコマンドへの応答かを識別できる。サーバーから状態変化を通知する場合は、先頭が `*` となり「タグなし応答」と呼ばれる。

IMAP4 で暗号化、認証を経て、メールサーバーのメールボックス `INBOX` にある 1 番目のメールのヘッダーを取得する場合、次のようなやり取りをする：

``` text
    foo.com                                                           example.com
┏━━━━━━┓                                                    ┏━━━━━━┓
┃            ┃                      * OK IMAP4rev1 Service Ready  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃  A001 CAPABILITY                                   ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃        * CAPABILITY IMAP4rev1 STARTTLS AUTH=PLAIN  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃                      A001 OK CAPABILITY completed  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃  A002 STARTTLS                                     ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃                 A002 OK Begin TLS negotiation now  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃クライアント┃┌────────────────────────┐┃  サーバー  ┃
┃            ┃│TLS ハンドシェイク開始                          │┃            ┃
┃            ┃│以降のやり取りは実際には TLS で暗号化されている │┃            ┃
┃            ┃└────────────────────────┘┃            ┃
┃            ┃  A003 CAPABILITY                                   ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃      * CAPABILITY IMAP4rev1 AUTH=PLAIN AUTH=LOGIN  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃                      A003 OK CAPABILITY completed  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃  A004 LOGIN user@example.com secret                ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃                           A004 OK LOGIN completed  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃  A005 SELECT INBOX                                 ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃                                        * 5 EXISTS  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃                                        * 0 RECENT  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃             A005 OK [READ-WRITE] SELECT completed  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃  A006 FETCH 1 BODY[HEADER]                         ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃                     * 1 FETCH (BODY[HEADER] {342}  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃┌────────────────────────┐┃            ┃
┃            ┃│1 番目のメールのヘッダーとして 342 バイトのデー │┃            ┃
┃            ┃│タが続き、閉じ括弧 ')' で終わる                 │┃            ┃
┃            ┃└────────────────────────┘┃            ┃
┃            ┃                           A006 OK FETCH completed  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃  A007 LOGOUT                                       ┃            ┃
┃            ┃―――――――――――――――――――――――――→┃            ┃
┃            ┃                    * BYE IMAP4 server logging out  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┃            ┃                          A007 OK LOGOUT completed  ┃            ┃
┃            ┃←―――――――――――――――――――――――――┃            ┃
┗━━━━━━┛                                                    ┗━━━━━━┛
```

IMAPS は、通信開始時から TLS で暗号化されるので、143 番ポートを使用できず（993 番ポートを使用する）、また、クライアントとサーバーの両方が対応していないと通信を開始できない。当然のことながら、IMAPS では `STARTTLS` コマンドは使わない。

電子メールの作成
----------------

標準ライブラリの `email` パッケージには、電子メールメッセージを扱う `email.message` モジュールと、便利な関数を提供する `email.utils` モジュールが含まれる。

`email.message.EmailMessage` クラスは、電子メールメッセージを表現する。コンストラクタは次のとおり。

``` python
email.message.EmailMessage(policy=default)
```

コンストラクタ引数 `policy` は、文字コードや行の折り返しなど電子メールの規約を定義する「ポリシーオブジェクト」を指定する。デフォルトでは、`email.policy.default` が使用される。これは、行末が `\r\n` ではなく、Python 標準の `\n` を使用すること以外は電子メールの標準規約 RFC 5322 に従う。

`email.policy.default` では、以下のような処理が行われる。

  * `Date` ヘッダーに設定する `datetime.datetime` オブジェクトは、RFC 5322 形式の文字列に変換される。
  * コンテンツがテキストなら、`'Content-Type: text/plain; charset="utf-8"'` と `'Content-Transfer-Encoding: 7bit'` が設定される。ただし、非 ASCII 文字を含む場合は、常に Base64 エンコードされ、`'Content-Transfer-Encoding: base64'` が設定される。
  * 本文の行の長さは最大 78 文字とされ、行が折り返される。

``` python
EmailMessage.set_content(content, maintype='text', subtype='plain', charset=None, language=None, disposition=None, filename=None, cid=None,
                         params=None, headers=None)
```

このメソッドは、本文やパートのコンテンツを設定する。

| 引数 | 意味 |
|:---|:---|
| `content` | 本文の内容。文字列ならテキスト、バイト列ならバイナリデータとして扱う |
| `maintype` | MIME タイプのメインタイプを文字列で指定。デフォルトは `'text'` |
| `subtype`	| MIME タイプのサブタイプを文字列で指定。デフォルトは `'plain'` |
| `charset` | エンコードに使う文字セット（`'utf-8'` など）を指定。`None`（デフォルト）なら `email.policy.default` では `'utf-8'` |
| `language` | `Content-Language` ヘッダーに使う言語タグを指定。`None`（デフォルト）なら `Content-Language` ヘッダーは付加されない |
| `disposition` | `Content-Disposition` ヘッダーを指定（`'inline'`, `'attachment'`）。`None`（デフォルト）なら `Content-Disposition` ヘッダーは付加されない |
| `filename` | 添付ファイルとして送信する場合のファイル名を指定。`Content-Disposition` ヘッダーの `filename` パラメータに使用される |
| `cid` | `Content-ID` ヘッダーの値。HTML メールで画像を埋め込む際などに使用。`None`（デフォルト）なら `Content-ID` ヘッダーは付加されない |
| `params` | `None`（デフォルト）としないなら、`Content-Type` ヘッダーに追加するパラメータを辞書で指定する |
| `headers` | `None`（デフォルト）としないなら、追加ヘッダーをタプル `(header_name, header_value)` のリストまたは辞書で指定する |

`content` 引数にバイト列を渡す場合、MIME タイプを `application/octet-stream` に設定する必要がある（そうでなければ `TypeError` 例外が発生）。コンテンツをテキストとして扱いたいなら文字列で渡すべきである。

``` python
EmailMessage.add_alternative(content, maintype='text', subtype='plain', charset=None, language=None, disposition=None, filename=None,
                             cid=None, params=None, headers=None)
```

このメソッドは、本体やパートを `'Content-Type: multipart/alternative'` と設定して、新しいパートを追加する。

通常、最初に `set_content()` メソッドでテキスト版を入れて、その後 `add_alternative()` で HTML 版などを追加する。

引数に関して `set_content()` メソッドと違う点は

  * `content` 引数にバイト列を渡すと、適切なエンコード（`base64` など）になり、`Content-Transfer-Encoding` ヘッダーが設定される。

``` python
EmailMessage.add_attachment(content, maintype, subtype, charset=None, language=None, disposition=None, filename=None, cid=None, params=None,
                            headers=None)
```

このメソッドは、本体やパートを `'Content-Type: multipart/mixed'` と設定して、指定したデータを添付ファイルパートとして追加する。

引数に関して `add_alternative()` メソッドと違う点は

  * `maintype` 引数と `subtype` 引数を省略できない。
  * `disposition` 引数を省略した場合、自動で `'Content-Disposition: attachment'` が付加される。

``` python
EmailMessage.add_related(content, maintype, subtype, charset=None, language=None, disposition=None, filename=None, cid=None, params=None,
                         headers=None)
```

このメソッドは、本体やパートを `'Content-Type: multipart/related'` と設定して、指定したデータを関連データパートとして追加する。主に HTML メールに画像やスタイルなどを埋め込みたいときに使う。

`cid` 引数を指定すると、HTML 内では `img` などの要素の `src` 属性を `'cid:XXX'` のように指定することでそのリソースを参照できる。`XXX` は `cid` 引数で指定した `Content-ID` ヘッダーの値（RFC 5322 形式なら山括弧 `<>` を除いたもの）。

引数に関して `add_attachment()` メソッドと違う点は

  * `disposition` 引数を省略した場合、自動で `'Content-Disposition: inline'` が付加される。

``` python
EmailMessage.is_multipart()
```

このメソッドは、メッセージがマルチパートの場合は `True` を返し、そうでない場合は `False` を返す。

``` python
EmailMessage.get_payload(i=None, decode=False)
```

メッセージがマルチパートの場合、このメソッドは、`i` が 0 から始まるインデックスなら子パートを表す `EmailMessage` オブジェクトを返す。`None`（デフォルト）の場合は各子パートを表す `EmailMessage` オブジェクトのリストを返す。

メッセージがシングルパートの場合、このメソッドは、`decode` が `True` なら `Content-Transfer-Encoding` ヘッダーに従ってコンテンツをデコードしたバイト列を返す。`False`（デフォルト）ならエンコードされたままの文字列を返す。

``` python
EmailMessage.as_string(unixfrom=False, maxheaderlen=None, policy=None)
```

このメソッドは、インスタンスの内容を平坦な文字列として返す。

| 引数 | 意味 |
|:---|:---|
| `unixfrom` | `True` の場合、エンベロープ From とエンベロープ To も戻り値に含まれる。デフォルトは `False` |
| `max_line_length` | 行の長さの最大を指定する。`None`（デフォルト）を指定すると、`email.policy.default` では 78 文字となる |

``` python
EmailMessage.__str__()
```

このメソッドは、`as_string()` メソッドを引数なしで呼び出し、その結果を返す。

``` python
EmailMessage.add_header(_name, _value, **_params)
```

このメソッドは、`_name` の名前を持つヘッダーフィールドを `_value` の値で追加する。`_value` が `None` の場合は、キーのみを追加する。パラメータをキーワード引数で指定できる。

すでに同一の名前で存在するフィールドは上書きされない。もしメッセージが名前 `_name` を持つフィールドを 1 つしか持たないようにしたければ、最初にそれを除去する必要がある。

``` python
EmailMessage.replace_header(_name, _value)
```

このメソッドは、メッセージ内で名前が `_name` に一致する最初のヘッダーフィールドを、もとのヘッダーの順序とフィールド名（大文字・小文字）を保持したままで、値が `_value` のものに置き換える。

もし一致するヘッダーが見つからない場合は、`KeyError` 例外を発生させる。

`email.message.EmailMessage` クラスは、`__getitem__()` などのメソッドもサポートし、添字構文 `[]` でヘッダーフィールドにアクセスできる。

  * 添字構文を評価すると、最初に一致するヘッダーフィールドの値を取得する。
  * 代入操作の左辺で添字構文を使用するときは、ヘッダーフィールドの追加となり、`add_header()` メソッドと同様となる。
  * del 文で添字構文を使用するときは、指定したヘッダーフィールドを除去する。

`email.message.EmailMessage` オブジェクトは、イテラブルであり、ヘッダーフィールドの名前を yield する。他のマッピングインターフェースもサポートしており、`in` や、`keys()`、`values()`、`items()`、`get()` メソッドを、ヘッダーフィールドの名前をキーとする辞書のように使用できる。

``` python
email.utils.make_msgid(idstring=None, domain=None)
```

`email.utils` モジュールが提供するこの関数は、RFC 2822 に準拠した `Message-ID` ヘッダーに適した文字列を返す。

| 引数 | 意味 |
|:---|:---|
| `idstring` | 戻り値の一意性を強化するために使用される文字列 |
| `domain` | `'@'` マークの後の部分（ドメイン部）を指定する。デフォルトではローカルホスト名が使用される |

次のコードは、シングルパートの電子メールメッセージを作成する例であり、`set_content()` メソッドを使用している。

In [ ]:
from email.message import EmailMessage
import datetime as dt

msg = EmailMessage()
msg["From"] = "from@example.com"
msg["To"] = "to@example.com"
msg["Subject"] = "テストメール"
msg["Date"] = dt.datetime.now(dt.timezone(dt.timedelta(hours=9)))
msg.set_content("これは Python から送信したメールです。")
print(msg)

From: from@example.com
To: to@example.com
Subject: テストメール
Date: Sun, 07 Sep 2025 17:21:27 +0900
Content-Type: text/plain; charset="utf-8"
Content-Transfer-Encoding: base64
MIME-Version: 1.0

44GT44KM44GvIFB5dGhvbiDjgYvjgonpgIHkv6HjgZfjgZ/jg6Hjg7zjg6vjgafjgZnjgIIK



次のコードは、画像ファイルを添付する電子メールメッセージを作成する例であり、`add_attachment()` メソッドを使用している。

In [ ]:
from email.message import EmailMessage
import datetime as dt
import mimetypes

msg = EmailMessage()
msg["From"] = "from@example.com"
msg["To"] = "to@example.com"
msg["Subject"] = "テストメール"
msg["Date"] = dt.datetime.now(dt.timezone(dt.timedelta(hours=9)))
msg.set_content("Vim icon")

# 画像ファイル
imgfile = "/usr/share/icons/locolor/16x16/apps/gvim.png"
maintype, subtype = mimetypes.guess_type(imgfile)[0].split('/')
with open(imgfile, "rb") as fp:
    img_data = fp.read()
msg.add_attachment(img_data, maintype, subtype, filename="gvim.png")
print(msg)

From: from@example.com
To: to@example.com
Subject: テストメール
Date: Sun, 07 Sep 2025 19:30:39 +0900
MIME-Version: 1.0
Content-Type: multipart/mixed; boundary="===============2004854929248673378=="

--===============2004854929248673378==
Content-Type: text/plain; charset="utf-8"
Content-Transfer-Encoding: 7bit

Vim icon

--===============2004854929248673378==
Content-Type: image/png
Content-Transfer-Encoding: base64
Content-Disposition: attachment; filename="gvim.png"
MIME-Version: 1.0

iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAMAAAAoLQ9TAAAAGFBMVEW9vb0A/wDAwMCAgIAAAID/
//8AgAAAAAAxi9wfAAAAAXRSTlMAQObYZgAAAFhJREFUeNpNzocBhDAMBEFHXf8do1uc5H9gh1hK
kaeKfXFPj4K9Rc3TJXZbuq9YvWD1BRqoAB2h7DCstijBI3rwXXSK4j7UiW6gPAL4zrWArrhgUeIL
G1kfBcgDtCKt4RYAAAAASUVORK5CYII=

--===============2004854929248673378==--



次のコードは、代替のプレーンテキストバージョン付きの HTML メッセージを作成する例であり、HTML に関連する画像を追加している。`add_alternative()` メソッドと `add_related()` メソッドを使用している。

In [ ]:
from email.message import EmailMessage
from email.utils import make_msgid
import datetime as dt
import mimetypes

msg = EmailMessage()
msg["From"] = "from@example.com"
msg["To"] = "to@example.com"
msg["Subject"] = "テストメール"
msg["Date"] = dt.datetime.now(dt.timezone(dt.timedelta(hours=9)))
msg.set_content("Vim icon")
vimimg_cid = make_msgid()
msg.add_alternative("""\
<html>
  <head></head>
  <body>
    <p>Vim icon</p>
    <img src="cid:{vimimg_cid}">
  </body>
</html>
""".format(vimimg_cid=vimimg_cid[1:-1]), subtype='html')

# 画像ファイル
imgfile = "/usr/share/icons/locolor/16x16/apps/gvim.png"
maintype, subtype = mimetypes.guess_type(imgfile)[0].split('/')
with open(imgfile, "rb") as fp:
    img_data = fp.read()
msg.get_payload()[1].add_related(img_data, maintype, subtype, cid=vimimg_cid)
print(msg)

From: from@example.com
To: to@example.com
Subject: テストメール
Date: Sun, 07 Sep 2025 19:33:03 +0900
MIME-Version: 1.0
Content-Type: multipart/alternative;
 boundary="===============6673116057823854676=="

--===============6673116057823854676==
Content-Type: text/plain; charset="utf-8"
Content-Transfer-Encoding: 7bit

Vim icon

--===============6673116057823854676==
MIME-Version: 1.0
Content-Type: multipart/related;
 boundary="===============1422449093306169437=="

--===============1422449093306169437==
Content-Type: text/html; charset="utf-8"
Content-Transfer-Encoding: 7bit

<html>
  <head></head>
  <body>
    <p>Vim icon</p>
    <img src="cid:175724118338.926.7131741058498778896@97026bad55dc">
  </body>
</html>

--===============1422449093306169437==
Content-Type: image/png
Content-Transfer-Encoding: base64
Content-ID: <175724118338.926.7131741058498778896@97026bad55dc>
MIME-Version: 1.0
Content-Disposition: inline

iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAMAAAAoLQ9TAAAAGFBMVEW9vb0A/wDAwMCAgIAAA

電子メールの送信
----------------

Python の標準ライブラリの `smtplib` モジュールは、作成した電子メールを SMTP などで送信する機能を提供する。

`smtplib.SMTP` クラスは、SMTP、SMTP-AUTH、STARTTLS を扱う。

``` python
smtplib.SMTP(host='', port=0, local_hostname=None, [timeout, ]source_address=None)
```

主な引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `host` | 電子メールを送る SMTP サーバーのホスト名。末尾がコロン `':'` の後に番号がついている場合は、「ホスト名:ポート番号」として扱われる |
| `port` | ポート番号 |
| `local_hostname` | `HELO`/`EHLO` コマンドのローカルホストの完全なドメイン名（FQDN）を指定する |
| `timeout` | 接続試行のようなブロッキング操作のタイムアウトを秒単位で指定する。指定しない場合は無制限となる |

`host` を指定して `smtplib.SMTP` インスタンスを作成すると、SMTP サーバーに接続しようとする。タイムアウトが切れると、`TimeoutError` 例外が送出される。

``` python
SMTP.set_debuglevel(level)
```

このメソッドは、デバッグ出力レベルを設定する。`level` が 1 や `True` の場合、接続ならびにサーバーとの送受信のメッセージがデバッグメッセージとなる。`level` が 2 の場合、それらのメッセージにタイムスタンプが付く。

``` python
SMTP.starttls(*, context=None)
```

このメソッドは、`STARTTLS` コマンドを送る。続く全ての SMTP コマンドが TLS で暗号化される。キーワード専用の `context` 引数は `ssl.SSLContext` オブジェクトを受け付け、デフォルトの高いセキュリティレベルの設定を変更するために使用される。通常、`context` を指定する必要はない。

このメソッドは、以下の例外を送出することがある:

  * `smtplib.SMTPHeloError`:  
サーバーが HELO に正しく返答しなかった。
  * `smtplib.SMTPNotSupportedError`:  
サーバーが STARTTLS に対応していない。
  * `RuntimeError`:  
実行中の Python インタープリターで、TLS サポートが利用できない。

``` python
SMTP.login(user, password, *, initial_response_ok=True)
```

このメソッドは、認証が必要な SMTP サーバーにログインする。認証に使用する引数は `user` と `password`。

``` python
SMTP.send_message(msg, from_addr=None, to_addrs=None, mail_options=(), rcpt_options=())
```

このメソッドは、メッセージを送信する。

| 引数 | 意味 |
|:---|:---|
| `msg` | `email.message.EmailMessage` オブジェクト |
| `from_addr` | 差出人のメールアドレス文字列。`None`（デフォルト）の場合、`msg` の `From` ヘッダーフィールドからアドレスを取得する |
| `to_addrs` | 宛先のメールアドレス文字列（のリスト）。`None`（デフォルト）の場合、`msg` の `To`、`Cc`、`Bcc` ヘッダーフィールド（もしあれば）からアドレスを取得す<br /><br />る |

このメソッドは、以下の例外を送出することがある:

  * `smtplib.SMTPRecipientsRefused`:  
全ての受信を拒否され、誰にもメールが届けられなかった。例外オブジェクトの `recipients` 属性は、受信拒否についての情報の入った辞書オブジェクトである。
  * `smtplib.SMTPHeloError`:  
サーバーが HELO に正しく返答しなかった。
  * `smtplib.SMTPSenderRefused`:  
サーバーが差出人アドレスを受理しなかった。
  * `smtplib.SMTPDataError`:  
サーバーが予期しないエラーコードを返した（受信拒否以外）。
  * `smtplib.SMTPNotSupportedError`:  
サーバーが UTF-8 をサポートしていない。

``` python
SMTP.quit()
```

SMTP セッションを終了し、接続を閉じる。

`smtplib.SMTP` はコンテキストマネージャーでもある。そのため with 構文をサポートする。`__enter__()` メソッドはオブジェクト自身を返す。`__exit__()` メソッドはオブジェクトの `quit()` メソッドを実行する。

SMTP サーバーの認証処理が不要な場合は

``` python
import smtplib

smtp_host = 'SMTPサーバーのホスト名'
smtp_port = 25

with smtplib.SMTP(smtp_host, smtp_port) as smtp:
    smtp.send_message(msg)
```

このコードだけでメール送信が処理される。

STARTTLS を使って暗号化された SMTP 通信で認証処理が必要な場合は、メール送信のコードは次のようになる。

``` python
import smtplib

smtp_host = 'SMTPサーバーのホスト名'
smtp_port = 587

# SMTP-AUTH に使用するユーザ名とパスワード
username = 'me@xxxx.com'
password = 'mypassword'

# メールサーバーへアクセスする
with smtplib.SMTP(smtp_host, smtp_port) as smtp:
    smtp.starttls()
    smtp.login(username, password)
    smtp.send_message(msg)
```

`smtplib.SMTP_SSL` クラスは SMTPS を扱う。

``` python
smtplib.SMTP_SSL(host='', port=0, local_hostname=None, *, [timeout, ]context=None, source_address=None)
```

キーワード専用の `context` 引数が追加されている。これは、`ssl.SSLContext` オブジェクトを受け付け、デフォルトの高いセキュリティレベルの設定を変更するために使用される。通常、`context` を指定する必要はない。

`smtplib.SMTP_SSL` は `smtplib.SMTP` のサブクラスであり、2 つのインスタンスは全く同じように動作する。

``` python
import smtplib

smtp_host = 'SMTPサーバーのホスト名'
smtp_port = 465

# SMTP-AUTH に使用するユーザ名とパスワード
username = 'me@xxxx.com'
password = 'mypassword'

# メールサーバーへアクセスする
with smtplib.SMTP_SSL(smtp_host, smtp_port) as smtps:
    smtps.login(username, password)
    smtps.send_message(msg)
```

Gmail の場合、STARTTLS にも、SMTPS にも対応している。ただし、事前に以下の準備が必要。

  * Google アカウントの管理ページを開く。
  * 「セキュリティ」に移動する。
  * 「2 段階認証プロセス」に移動する。
  * 2 段階認証プロセスを有効にする。
  * 「アプリ パスワード」で適当にアプリパスワードを 1 つ作成する（アプリ名は何でもよい）。

`login` メソッドに渡すパスワードには、アプリパスワードを渡す必要がある。

``` python
smtp_host = 'smtp.gmail.com'
smtp_port = 587  # SMTPS なら 465
username = 'xxxx@gmail.com'
password = 'アプリパスワード'
```

電子メールの解析
----------------

`email` パッケージは、ファイルや文字列からメッセージオブジェクト構造を作成する関数を提供している。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `email.message_from_bytes(s, _class=None, *, policy=policy.compat32)` | バイト列からメッセージオブジェクト構造を作成して返す | `Message` |
| `email.message_from_binary_file(fp, _class=None, *, policy=policy.compat32)` | 開かれたバイナリファイルのファイルオブジェクトからメッ<br /><br />セージオブジェクト構造を作成して返す | `Message` |
| `email.message_from_string(s, _class=None, *, policy=policy.compat32)` | 文字列からメッセージオブジェクト構造を作成して返す | `Message` |
| `email.message_from_file(fp, _class=None, *, policy=policy.compat32)` | 開かれたテキストファイルのファイルオブジェクトからメッ<br /><br />セージオブジェクト構造を作成して返す | `Message` |

これらの関数の戻り値の型 `email.message.Message` は、`email.message.EmailMessage` クラスの基底クラスであり、MIME を処理する機能のない簡素なものである。パーサーのメソッドが  `email.message.EmailMessage` のインスタンスを返すようにするには、コンストラクタ引数 `policy` を `email.policy.default` に指定する必要がある。

In [ ]:
import email
import email.policy

headers = email.message_from_string(
    "From: Foo Bar <user@example.com>\n"
    "To: <someone_else@example.com>\n"
    "Subject: Test message\n"
    "\n"
    "Body would go here\n",
    policy=email.policy.default
)

#  headers は EmailMessage のインスタンス
headers["to"], format(headers["from"]), headers["subject"]

('someone_else@example.com', 'Foo Bar <user@example.com>', 'Test message')

電子メールの取得
----------------

標準ライブラリの `imaplib` モジュールは、メールサーバー上の電子メールを IMAP4 などで取得する機能を提供する。

`imaplib.IMAP4` クラスは、IMAP4 を扱う。

``` python
imaplib.IMAP4(host='', port=IMAP4_PORT, timeout=None)
```

| 引数 | 意味 |
|:---|:---|
| `host` | メールサーバーのドメイン名 |
| `port` | ポート番号 |
| `timeout` | 接続試行のようなブロッキング操作のタイムアウトを秒単位で指定する。指定しない場合は無制限となる |

`imaplib.IMAP4` のインスタンスは、以下のメソッドを持つ。これらのメソッドは `(応答コード, データ)` のタプルを返す。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `starttls(ssl_context=None)` | `STARTTLS` コマンドを送る。`ssl_context` 引数は、`ssl.SSLContext` オブジェクトで、デフォルトの<br /><br />高いセキュリティレベルの設定を変更するために使用される。通常は指定不要 | `tuple` |
| `login(user, password)` | 認証が必要な IMAP4 サーバーにログインする。認証に使用する引数は `user` と `password` | `tuple` |
| `logout()` | サーバーへの接続を遮断する | `tuple` |
| `select(mailbox='INBOX', readonly=False)` | メールボックスを選択する。戻り値タプルの第 2 要素は `mailbox` 内のメッセージ数。デフォルトでは<br /><br /> `'INBOX'` を選択する。`readonly` が設定された場合、メールボックスに対する変更はできない | `tuple` |
| `list([directory[, pattern]])` | `pattern` にマッチする `directory` メールボックス名を列挙する。`directory` のデフォルトは最上<br /><br />レベルのメールフォルダで、`pattern` のデフォルトは全てにマッチする。戻り値タプルの第 2 要素は<br /><br /> `LIST` コマンドの応答のリスト | `tuple` |
| `search(charset, *criteria)` | 選択されているメールボックスを検索する。`charset` で検索文字列のエンコーディングを指定する。<br /><br />通常は `None` を指定する（ほとんどのサーバーがサポートする UTF8 でエンコーディングされる）。<br /><br />`criteria` で IMAP4 `SEARCH` コマンドと同様の検索条件を文字列で指定する。戻り値タプルの第 2<br /><br /> 要素は検索結果のメール ID（UID）がスペース区切りのバイト文字列として格納されたもの | `tuple` |
| `fetch(message_set, message_parts)` | メール（の一部）を取得する。`message_parts` は `FETCH` コマンドで使用可能な「取得データ種類」を<br /><br />表す文字列を丸括弧で囲ったもの。戻り値タプルの第 2 要素はメッセージパートのエンベロープ情報<br /><br />とデータからなるタプル | `tuple` |
| `store(message_set, command, flag_list)` | 選択されているメールボックス内のメッセージ群のフラグ設定を変更する。`command` は `'FLAGS'`,<br /><br /> `'+FLAGS'`, `'-FLAGS'` のいずれか。オプションで末尾に `'.SILENT'` が付くこともある | `tuple` |
| `expunge()` | 選択されているメールボックスから削除フラグ付きメールを完全に削除する | `tuple` |

`imaplib.IMAP4` のインスタンスは、以下の属性を持つ。

| 属性 | 意味 |
|:---|:---|
| `debug` | デバッグ出力レベルを設定する |
| `state` | オブジェクトが現在置かれている状態を示す以下の文字列<br /><br />・`'NONAUTH'`: オブジェクトがインスタンス化された直後の状態。まだサーバーとの認証が完了していない<br /><br />・`'AUTH'`: ユーザー認証が成功した後の状態。この状態では、ユーザーはサーバー上のメールボックスを操作できるが、特定のメールボックスはまだ<br /><br />　選択されていない<br /><br />・`'SELECTED'`: `select()` メソッドが正常に実行され、特定のメールボックスが選択された後の状態。この状態では、選択されたメールボックス内<br /><br />　のメッセージを操作できる<br /><br />・`'LOGOUT'`: `logout()` メソッドが実行され、サーバーから正常にログアウトした後の状態 |

`imaplib.IMAP4_SSL` クラスは IMAPS を扱う。

``` python
imaplib.IMAP4_SSL(host='', port=IMAP4_SSL_PORT, *, ssl_context=None, timeout=None)
```

キーワード専用の `ssl_context` 引数が追加されている。これは、`ssl.SSLContext` オブジェクトを受け付け、デフォルトの高いセキュリティレベルの設定を変更するために使用される。通常、`ssl_context` を指定する必要はない。

`imaplib.IMAP4_SSL` は `imaplib.IMAP4` のサブクラスであり、2 つのインスタンスは全く同じように動作する。

Gmail の場合、143 番ポートをブロックするので、`imaplib.IMAP4_SSL` インスタンスと 993 番ポートを使用する必要がある。メール送信と同様に「2 段階認証プロセスの有効化」と「アプリパスワードの作成」も必要。

``` python
import imaplib
import email
import email.policy

# 設定情報
IMAP_SERVER = "imap.gmail.com"
IMAP_PORT = 993
GMAIL_USER = "メールアドレス"
APP_PASSWORD = "アプリパスワード"


def get_unread_emails(limit=5):
    """Gmailから未読メールを最大limit件取得して表示する"""
    try:
        # IMAPサーバーにSSL接続
        print("IMAPサーバーに接続中...")
        imap = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)

        # ログイン
        print("ログイン中...")
        imap.login(GMAIL_USER, APP_PASSWORD)

        # INBOXフォルダを選択（読み取り専用）
        imap.select("INBOX", readonly=True)

        # 未読メールを検索
        status, messages = imap.search(None, "(UNSEEN)")

        # 未読メールがなければ終了
        if status != "OK" or not messages[0]:
            print("未読メールはありません。")
            return

        email_ids = messages[0].split()
        print(f"未読メールが{len(email_ids)}件見つかりました。")

        email_ids.reverse()

        if len(email_ids) > limit:
            email_ids = email_ids[:limit]
            print(f"最大{limit}件まで表示します。")

        for e_id in email_ids:
            res, msg_data = imap.fetch(e_id, "(RFC822)")
            if res != "OK":
                continue

            # メールデータを解析
            raw_email = msg_data[0][1]
            msg = email.message_from_bytes(raw_email, policy=email.policy.default)

            # ヘッダー情報のデコード
            subject = msg["Subject"]
            sender = msg["From"]

            print("-" * 20)
            print(f"送信元: {sender}")
            print(f"件名: {subject}")

    except Exception as e:
        print(f"エラーが発生しました: {e}")

    finally:
        # IMAP接続を閉じる
        if "imap" in locals() and imap.state == "AUTH":
            imap.logout()
            print("ログアウトしました。")


if __name__ == "__main__":
    get_unread_emails()
```